In [1]:
import parser
from parser import IRParser

from ir import Var, Const, Seq, If, FuncDef


In [2]:
ret_loc_count = 0
end_count = 0 
else_count =0


def modify_global_else():
    global else_count
    else_count = else_count + 1

def modify_global_end():
    global end_count
    end_count = end_count + 1
    
def modify_global_ret():
    global ret_loc_count
    ret_loc_count = ret_loc_count + 1
    
    
def compilerFunc():
    p = IRParser()
   # with open(filename, 'r') as file:
    #    file_contents = file.read()
    
    #a b y x
    pList = p('''

main(0) = 0x4d; 1; 0xa808; 0xa800; 0x0; 0x0; 0x0; 0x0; 0x0; 0x0; 0x0; 0x0; 0x0; 0x0; 0x0;
            init; wait; if $-16 - 0x4d then (if $-16 - 0x4b then (if $-16 - 0x48 then (if $-16 - 0x50 then game else setCurrDirection $-16; game)
            else setCurrDirection $-16; game) else setCurrDirection $-16; game) else game

init(0) = mem_poke 0xffff 0xa800; mem_poke 0xff 0xa808; 1

game(0) = if move then (wait; if $-1 - 0x4d then (if $-1 - 0x4b then (if $-1 - 0x48 then (if $-1 - 0x50 then .game else setCurrDirection $-1; .game)
                    else setCurrDirection $-1; .game) else setCurrDirection $-1; .game) else setCurrDirection $-1; .game) else 0

move(0) = getCurrDirection; getHead; getAppleLoc;
if $-1 - 0x4d then (
            if $-1 - 0x4b then (
                if $-1 - 0x48 then (
                    if $-1 - 0x50 then 0
                        else (if checkHeadDown $-2 then (if $-2 + 0x10 - $-3 then moveWithoutApple ($-2 + 0x10) else moveWithApple ($-2 + 0x10)) else 0)
                    )
                    else if checkHeadUp $-2 then (if $-2 - 0x10 - $-3 then moveWithoutApple ($-2 - 0x10) else moveWithApple ($-2 - 0x10)) else 0

                )
                else if checkHeadLeft $-2 0xa000 then (if $-2 - 0x1 - $-3 then moveWithoutApple ($-2 - 0x1) else moveWithApple ($-2 - 0x1)) else 0
            )
            else if checkHeadRight $-2 0xa00f then (if $-2 + 0x1 - $-3 then moveWithoutApple ($-2 + 0x1) else moveWithApple ($-2 + 0x1)) else 0

checkHeadRight(2) = if $1 < 0xb000 then (if $0 - $1 then .checkHeadRight $0 ($1 + 0x10) else 0) else 1

checkHeadLeft(2) = if $1 < 0xb000 then (if $0 - $1 then .checkHeadRight $0 ($1 + 0x10) else 0) else 1

checkHeadUp(1) = if $0 < 0xa000 then 1 else (if 0xa00f < $0 then 1 else 0)

checkHeadDown(1) = if $0 < 0xaff0 then 1 else (if 0xafff < $0 then 1 else 0)

moveWithoutApple(1) = getLength; if checkBody $0 ($-2 - 1) then updateSnake $0 $-2; drawSnake; mem_poke 0x0 (mem_peek (4 + $-2)); 1 else 0

moveWithApple(1) = getLength; if $-2 < 10 then (if checkBody $0 $-2 then updateSnake $0 $-2; drawSnake; setLength($-2 + 1); updateApple; 1 else 0) else 0

checkBody(2) = if 0 < $1 then (if (mem_peek (3 + $1)) - $0 then checkBody $0 ($1 - 1) else 0) else 1

updateSnake(2) = if 0 < $1 then mem_poke (mem_peek (3 + $1)) (4 + $1); updateSnake $0 ($1 - 1) 
                         else mem_poke $0 0x4

drawSnake(0) = getCurrDirection; getHead; if $-1 - 0x4d then if $-1 - 0x4b then if $-1 - 0x48 then if $-1 - 0x50 then 0
                    else mem_poke 0xff $-2
                    else mem_poke 0xff $-2
                    else mem_poke 0xffff $-2
                    else mem_poke 0xffff $-2

updateApple(0) = (0xa000 + computeRandom (getAppleLoc - 0xa000)); getLength; if checkBody $-1 ($-2 - 1) then mem_poke $-1 0x3; mem_poke 0xff $-1 else .updateApple


computeRandom(1) = mod (($0 * 0x1ff) + 0x1f0) 0xfff

mod(2) = if $0 < $1 then $0 else .mod ($0 - $1) $1

wait(0) = mem_peek 0xc001; wait_aux

wait_aux(0) = 
    if (mem_peek 0xc001) - $1 /* < access closure */
    then mem_peek 0xc000 else .wait_aux

getCurrDirection(0) = mem_peek 0x1
setCurrDirection(1) = mem_poke $0 0x1

getLength(0) = mem_peek 0x2
setLength(1) = mem_poke $0 0x2

getAppleLoc(0) = mem_peek 0x3

getHead(0) = mem_peek 0x4

mem_peek(1) = 1
mem_poke(2) = 1

    ''')
    #print(pList)
    # p_list contains list of funcDef: [funcDef_1, funcDef_2, funcDef_3, ...., funcDef_n]
    to_return = []
    tmp_instr = []
    counter = 0
    to_return.append(('PUSH', 'finish'))
    # maybe we need to append ('JMP', 'main')
    #print(pList)
    for funcDef in pList:
        if funcDef.name == 'mem_peek':
            tmp_instr.append(funcDef.name)
            tmp_instr.append(('DUP', 0))
            tmp_instr.append(('POP', 1))
            tmp_instr.append(('LOAD', 0))
            tmp_instr.append(('YANK', (1, 1)))
            tmp_instr.append(('POP', 2))
            tmp_instr.append(('RET', 1))
            to_return = to_return + tmp_instr
            tmp_instr = []
            continue
            
        if funcDef.name == 'mem_poke':
            tmp_instr.append(funcDef.name)
            tmp_instr.append(('DUP', 1))
            tmp_instr.append(('DUP', 1))
            tmp_instr.append(('POP', 2))
            tmp_instr.append(('STOR', 0))
            tmp_instr.append(('YANK', (1, 1)))
            tmp_instr.append(('POP', 2))
            tmp_instr.append(('RET', 1))
            to_return = to_return + tmp_instr
            tmp_instr = []
            continue
            
        tmp_instr.append(funcDef.name)
        counter = funcDef.nargs
        tmp_instr, counter = compilerFunc_aux(funcDef.body, tmp_instr, counter, funcDef.nargs)
        if tmp_instr == None:
            continue
        #counter = counter + 1
        if counter >= 2:
            tmp_instr.append(('YANK', (1, counter - 1)))
            tmp_instr.append(('POP', 2))
            tmp_instr.append(('RET', 1))
        elif counter >= 1:
            tmp_instr.append(('POP', 2))
            tmp_instr.append(('RET', 1))
        elif counter == 0:
            tmp_instr.append(('POP', 1))
            tmp_instr.append(('RET', 0))
        to_return = to_return + tmp_instr
        tmp_instr = []
    to_return.append('finish')
    to_return.append(('POP', 1))
    to_return.append(('HALT', 0))
    return to_return

def compilerFunc_aux(node, to_return, counter, numArgs):
    
    if isinstance(node, Const):
        to_return.append(('PUSH', node[0]))
        counter = counter + 1
        return to_return, counter
            
    elif isinstance(node, Var):
        if node[0] < numArgs and node[0] >= 0:
            offs = counter - node[0] - 1
        else: offs = node[0] + counter
        to_return.append(('DUP', offs))
        counter = counter + 1
        return to_return, counter
            
            
    elif isinstance(node, If):
        cond = node[0]
        to_return, counter = compilerFunc_aux(cond, to_return, counter, numArgs)
        to_return.append(('POP', 1))
        counter = counter - 1
        _else_loc = '_else_' + str(else_count)
        #else_count = else_count + 1
        modify_global_else()
        to_return.append(('JZ', _else_loc))
        then = node[1]
        to_return, tmp = compilerFunc_aux(then, to_return, counter, numArgs)
        if tmp - counter > 1:
            to_return.append(('YANK', (1, tmp - counter - 1)))
        _end_loc = '_end_' + str(end_count)
        #end_count = end_count + 1
        modify_global_end()
        to_return.append(('JMP', _end_loc))
        els = node[2]
        to_return.append(_else_loc)
        to_return, tmp = compilerFunc_aux(els, to_return, counter, numArgs)
        if tmp - counter > 1:
            to_return.append(('YANK', (1, tmp - counter - 1)))
        to_return.append(_end_loc)
        counter = counter + 1
        return to_return, counter
            
    elif isinstance(node, Seq):
        for e in node[0]:
            to_return, counter = compilerFunc_aux(e, to_return, counter, numArgs)
        return to_return, counter
        
    else:
        if node[0] == '-':
            if len(node) == 2:
                to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
                to_return.append(('POP', 1))
                to_return.append(('ALU', 'NEG'))
                return to_return, counter
            
            elif len(node) == 3:
                to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
                to_return, counter = compilerFunc_aux(node[2], to_return, counter, numArgs)
                to_return.append(('POP', 2))
                to_return.append(('ALU', 'SUB'))
                counter = counter - 1
                return to_return, counter
            
        elif node[0] == '~':
            to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
            to_return.append(('POP', 1))
            to_return.append(('ALU', 'NOT'))
            return to_return, counter
        
        elif node[0] == '*':
            to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
            to_return, counter = compilerFunc_aux(node[2], to_return, counter, numArgs)
            to_return.append(('POP', 2))
            to_return.append(('ALU', 'MUL'))
            counter = counter - 1
            return to_return, counter
            
        elif node[0] == '+':
            to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
            to_return, counter = compilerFunc_aux(node[2], to_return, counter, numArgs)
            to_return.append(('POP', 2))
            to_return.append(('ALU', 'ADD'))
            counter = counter - 1
            return to_return, counter
            
        elif node[0] == '<':
            to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
            to_return, counter = compilerFunc_aux(node[2], to_return, counter, numArgs)
            to_return.append(('POP', 2))
            to_return.append(('ALU', 'LT'))
            counter = counter - 1
            return to_return, counter
        
        elif node[0] == '<<':
            to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
            to_return, counter = compilerFunc_aux(node[2], to_return, counter, numArgs)
            to_return.append(('POP', 2))
            to_return.append(('ALU', 'SHL'))
            counter = counter - 1
            return to_return, counter
        
        elif node[0] == '>>':
            to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
            to_return, counter = compilerFunc_aux(node[2], to_return, counter, numArgs)
            to_return.append(('POP', 2))
            to_return.append(('ALU', 'SHR'))
            counter = counter - 1
            return to_return, counter
        
        elif node[0] == '|':
            to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
            to_return, counter = compilerFunc_aux(node[2], to_return, counter, numArgs)
            to_return.append(('POP', 2))
            to_return.append(('ALU', 'OR'))
            counter = counter - 1
            return to_return, counter
        
        elif node[0] == '&':
            to_return, counter = compilerFunc_aux(node[1], to_return, counter, numArgs)
            to_return, counter = compilerFunc_aux(node[2], to_return, counter, numArgs)
            to_return.append(('POP', 2))
            to_return.append(('ALU', 'AND'))
            counter = counter - 1
            return to_return, counter
        
        elif node[0] == 'ignore':
            return to_return, counter
            
        elif node[0] == '@':
            _ret_loc = '_ret_loc_' + str(ret_loc_count)
            #ret_loc_count = ret_loc_count + 1
            modify_global_ret()
            old_counter = counter
            to_return.append(('PUSH', _ret_loc))
            counter = counter + 1
            new_counter = counter
            if (len(node) > 2):
                for i in range(2, len(node)):
                    to_return, new_counter = compilerFunc_aux(node[i], to_return, new_counter, numArgs)
                counter = new_counter - counter 
            else: counter = 0
            to_return.append(('JMP', node[1]))
            to_return.append(_ret_loc)
            counter = old_counter + 1
            return to_return, counter
        
        elif node[0] == '@.':
            old_counter = counter
            new_counter = counter
            if (len(node) > 2):
                for i in range(2, len(node)):
                    to_return, new_counter = compilerFunc_aux(node[i], to_return, new_counter, numArgs)
                counter = new_counter - counter 
            else: counter = 0
            to_return.append(('YANK', (counter, old_counter)))
            to_return.append(('JMP', node[1]))
            #return_value = 1
            #counter = counter + return_value
            return to_return, counter

In [3]:

try1 = compilerFunc()
print(try1)

[('PUSH', 'finish'), 'find_max', ('PUSH', '_ret_loc_0'), ('DUP', 2), ('DUP', 2), ('PUSH', 1), ('PUSH', '_ret_loc_1'), ('DUP', 6), ('JMP', 'mem_peek'), '_ret_loc_1', ('JMP', 'find_max_aux'), '_ret_loc_0', ('YANK', (1, 2)), ('POP', 2), ('RET', 1), 'find_max_aux', ('DUP', 1), ('DUP', 3), ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_0'), ('DUP', 0), ('PUSH', '_ret_loc_2'), ('DUP', 5), ('DUP', 4), ('POP', 2), ('ALU', 'ADD'), ('JMP', 'mem_peek'), '_ret_loc_2', ('POP', 2), ('ALU', 'LT'), ('POP', 1), ('JZ', '_else_1'), ('DUP', 3), ('DUP', 3), ('DUP', 3), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('PUSH', '_ret_loc_3'), ('DUP', 7), ('DUP', 6), ('POP', 2), ('ALU', 'ADD'), ('JMP', 'mem_peek'), '_ret_loc_3', ('YANK', (4, 4)), ('JMP', 'find_max_aux'), ('JMP', '_end_0'), '_else_1', ('DUP', 3), ('DUP', 3), ('DUP', 3), ('PUSH', 1), ('POP', 2), ('ALU', 'ADD'), ('DUP', 3), ('YANK', (4, 4)), ('JMP', 'find_max_aux'), '_end_0', ('JMP', '_end_1'), '_else_0', ('DUP', 0), '_end_1', ('YANK', (1, 4)), ('